<a href="https://colab.research.google.com/github/jasonwong-lab/HKU_STEM/blob/main/HKUSTEM23%20_SARS_CoV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### How sequencing is used to identify SARS-CoV2 strains (HKU STEM Bootcamp on Biomedical data science)

*by Dr Jason Wong (Associate Professor, School of Biomedical Sciences, HKU)*

The objective of this workshop is show demonstrate how sequencing data of viruses are used to identify coronavirus strains and to study the effect of mutations on the virus.


## Set working directory

By default working directoy will be My Drive/SARS-CoV2

In [ ]:
# set working pathway to your own google drive doc (~ 1 min)
from google.colab import drive
drive.mount('/content/gdrive')                         # if using for the first time, you be requested to grant permission to link your Google Drive

import os
try:
  os.mkdir("/content/gdrive/My Drive/SARS-CoV2")         # change this path if necessary
except FileExistsError:
  print("directory already exist. OK to continue")
os.chdir("/content/gdrive/My Drive/SARS-CoV2")

In [ ]:
pwd

## Package installation and downloads for workshop (~ 5 minutes)

1.   conda (for simple installation of packages)
2.   ClustalW (for sequence alignment)
3.   biopython (tools for interacting with bioinformatics file formats)
4.   py3Dmol (for protein structure visualisation)
5.   Download ready prepared files for analysis.


In [ ]:
# install conda (~ 1 min). There will be a message saying that the session has crashed but don't worry about this. This is due to the session restarting following conda installation
!pip install -q condacolab
import condacolab#
condacolab.install()

In [ ]:
# install ClustalW
!conda install -c bioconda clustalw

In [ ]:
# install Biopython
# note that there maybe a message about the need for a restart due to a new version of numpy. You can ignore this.
!pip install biopython

In [ ]:
# install py3Dmol
!pip install py3Dmol


In [ ]:
# double check that we are in right directory
import os
os.chdir("/content/gdrive/My Drive/SARS-CoV2")

# download fasta files for analysis
!wget -O SARSCoV2_Spike.fa https://raw.githubusercontent.com/jasonwong-lab/HKU_STEM/main/SARSCoV2_Spike.fa       # SARSCov2_Spike.fa
!wget -O Virus_seq.fa https://raw.githubusercontent.com/jasonwong-lab/HKU_STEM/main/Virus_seq.fa                 # Virus_seq.fa

In [ ]:
ls

## Whole genome alignment of coronaviruses

We have prepared sequences for 5 well characterised coronavirus species. These are:

*   SARS-CoV2
*   SARS-CoV
*   AlphaCoV 229E
*   AlphaCoV NL63
*   MERS

For this first section, we will demonstrate how the genome sequence of the viruses can be used to distingush and reveal their relationship.

1.   Run multiple sequence alignment on the virus sequences
2.   Count the number of differing nucleotides between SARS-CoV2 and the other virues
3.   Construct a phylogenetic tree



In [ ]:
# take a look at Fasta file format
!head Virus_seq.fa

In [ ]:
# run ClustalW for alignment. (~3 minutes)
from Bio import AlignIO
from Bio.Align.Applications import ClustalwCommandline
cmd = ClustalwCommandline("clustalw2",infile="Virus_seq.fa",topdiags=1,quicktree=1)
print(cmd)
stdout, stderr = cmd()

# check to see output files generated by ClustalW. Should see two new files Virus_seq.aln and Virus_seq.dnd
!ls

In [ ]:
# Display alignment result as text output. First 50 lines.
align = open("Virus_seq.aln", "r")
print(''.join(align.readlines()[0:100]))

In [ ]:
# Count number of nucleotides that are different between SARS-CoV2 and each virus
# read alignment result
from Bio import AlignIO
align = AlignIO.read("Virus_seq.aln", "clustal")
print(align)

# Get SARSCoV2 record
SARSCoV2 = align[0]

# Loop through and count differences
for i in align:
  diff_nuc = 0
  pos = 0
  for j in i.seq:
    if (j != "-" or SARSCoV2.seq[pos] != "-") and j != SARSCoV2.seq[pos]:
       diff_nuc+=1
    pos+=1
  print(i.id, diff_nuc)

In [ ]:
#Draw phylogenetic tree

from Bio import Phylo
tree = Phylo.read("Virus_seq.dnd", "newick")
Phylo.draw_ascii(tree)

## Spike protein sequence analysis

The spike protein is of considerable interest in terms of SARS-CoV2 because it is responsible for recognising human cells and mediating entry and infection. Furthermore, COVID-19 vaccines are designed to generate antibodies that target the spike protein. Therefore mutations in the spike protein can both affect the virulance of SARS-CoV2 and may also affect the efficacy of vaccines.

For this part of the workshop we will specifically analyse the spike protein of SARS-CoV2 strains. Specifically we will:

1.   Translate the S gene into the spike protein sequence.
2.   Align and compare the spike protein for SARS-CoV2 strains.
3.   Find the specific amino acid differences between Wuhan strain with Delta and Omicron BA.5
4.   Visualise the spike protein structure.



In [ ]:
# Extract S gene and translate to protein sequence
# Based on NCBI the location of the Spike gene is between 21563..25384
from Bio import SeqIO
rawSeq = SeqIO.parse("Virus_seq.fa", "fasta")
for i in rawSeq:
  if i.id == "SARSCoV2_NC045512":
    print(i)
    Sgene = i.seq[21562:25384]              # python index are 0-based so need to subtract start by 1. End is exclusive so leave as is.

print("\nSARSCoV2_NC045512 S gene sequence")
print(Sgene)

#translate from DNA to Protein
Sprotein = Sgene.translate()
print("\nSARSCoV2_NC045512 spike protein sequence")
print(Sprotein)


In [ ]:
# run Clustalw and again aligned amino acid sequences with the same length
from Bio.Align.Applications import ClustalwCommandline
cmd = ClustalwCommandline("clustalw2",infile="SARSCoV2_Spike.fa")
print(cmd)
stdout, stderr = cmd()

# look at phylogenetic relationship
from Bio import Phylo
tree = Phylo.read("SARSCoV2_Spike.dnd", "newick")
Phylo.draw_ascii(tree)

In [ ]:
# Display alignment result as text output (show only first 98 lines of the file).
align = open("SARSCoV2_Spike.aln", "r")
print(''.join(align.readlines()[0:50]))

In [ ]:
#Get differences in Spike protein between Wuhan and Delta
#read alignment result
from Bio import AlignIO
align = AlignIO.read("SARSCoV2_Spike.aln", "clustal")

print ("Delta variants in Spike protein")

#Get Wuhan record
Wuhan = align[0]

#Loop through and find mutations in Delta Spike protein
for i in align:
  if i.id == "Delta":
    pos = 0
    realpos = 0             # accounts for gaps in the Wuhan protein sequence
    for j in i.seq:
      if j != Wuhan.seq[pos]:
        print(Wuhan.seq[pos],str(realpos+1),j)
      if (Wuhan.seq[pos] != "-"):
          realpos+=1
      pos+=1


print ("\nOmicron BA.5 variants in Spike protein")
#Loop through and find mutations in Omicron BA.5 Spike protein
for i in align:
  if i.id == "Omicron.BA5":
    pos = 0
    realpos = 0             # accounts for gaps in the Wuhan protein sequence
    for j in i.seq:
      if j != Wuhan.seq[pos]:
        print(Wuhan.seq[pos],str(realpos+1),j)
      if (Wuhan.seq[pos] != "-"):
          realpos+=1
      pos+=1


In [ ]:
# Visualising the structure of the spike protein
# load cyroEM structure of spike protein from PDB (protein databank)

import py3Dmol
p = py3Dmol.view(query='mmtf:7BBH',width=600,height=600)
p.setBackgroundColor('white')
p.setStyle({'cartoon': {'color':'spectrum'}})
p.addStyle({'chain':'A'},{'cartoon': {'color':'red'}})
p.addStyle({'chain':'B'},{'cartoon': {'color':'green'}})
p.addStyle({'chain':'C'},{'cartoon': {'color':'blue'}})

p.show()

In [ ]:
#view RBD (receptor bind domain variants)
p = py3Dmol.view(query='mmtf:7BBH',width=600,height=600)
p.setBackgroundColor('white')
p.setStyle({'cartoon': {'color':'spectrum'}})
p.addStyle({'chain':'C','resi':["319-541"]},{'cartoon': {'color':'red'}})
p.addStyle({'chain':'C','resi':[452,478]},{'stick':{'colorscheme':'skyBlueCarbon'}})
p.show()